# 🎬 VideoRobot — نوت‌بوک نهایی برای اجرا در Colab
این نوت‌بوک ریپازیتوری را کلون کرده، وابستگی‌ها را نصب می‌کند، بک‌اند Flask را اجرا می‌کند، یک درخواست رندر نمونه ارسال می‌کند و ویدیوی نهایی را دانلود می‌کند.

In [ ]:
# 🧠 Mount Google Drive for assets
from google.colab import drive
import pathlib

try:
    drive.mount('/content/drive')
    print('✅ Google Drive mounted.')
    DRIVE_ASSETS = pathlib.Path('/content/drive/MyDrive/VideoRobot/Assets')
    DRIVE_OUTPUT = pathlib.Path('/content/drive/MyDrive/VideoRobot/Output')
    DRIVE_ASSETS.mkdir(parents=True, exist_ok=True)
    DRIVE_OUTPUT.mkdir(parents=True, exist_ok=True)
    print(f'✅ Assets directory: {DRIVE_ASSETS}')
except Exception as e:
    print(f'ℹ️ Could not mount Drive: {e}')
    DRIVE_ASSETS = None
    DRIVE_OUTPUT = None

In [ ]:
# 📦 Clone repository fresh
import shutil, pathlib, subprocess, sys

REPO_URL = "https://github.com/englishpodcasteasy-glitch/videorobot.git"
WORK = pathlib.Path('/content/videorobot')
BACKEND_DIR = WORK / 'backend'

print('🗑️ Cleaning up old repo...')
shutil.rmtree(WORK, ignore_errors=True)

print('📥 Cloning repository...')
subprocess.run(['git', 'clone', REPO_URL, str(WORK)], check=True)
print(f'✅ Repo cloned at {WORK}')

# اضافه کردن مسیر پروژه به sys.path برای ایمپورت‌ها
if str(BACKEND_DIR) not in sys.path:
    sys.path.insert(0, str(BACKEND_DIR))

In [ ]:
# ⚙️ Install FFmpeg and Python dependencies
print('🔧 Installing FFmpeg...')
!apt-get update -qq && apt-get install -y ffmpeg

print('📦 Installing Python dependencies...')
requirements_path = BACKEND_DIR / 'requirements.txt'
if requirements_path.exists():
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-r', str(requirements_path)], check=True)
    print('✅ Dependencies installed.')
else:
    print('⚠️ requirements.txt not found, skipping pip install.')

In [ ]:
# 🚀 Start backend server and wait for it to be healthy
import os, time, requests, subprocess, logging

# تنظیم لاگ برای دیدن خروجی‌های بک‌اند در Colab
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# تنظیم متغیرهای محیطی برای بک‌اند
os.environ['VR_HOST'] = '0.0.0.0'
os.environ['VR_PORT'] = '8000'

PORT = os.environ['VR_PORT']
BASE_URL = f"http://127.0.0.1:{PORT}"
HEALTH_URL = f"{BASE_URL}/health"

# مسیر فایل اصلی بک‌اند
MAIN_SCRIPT = BACKEND_DIR / 'main.py'

if not MAIN_SCRIPT.exists():
    raise FileNotFoundError(f"Main script not found at {MAIN_SCRIPT}")

print(f"🚀 Starting backend server: {MAIN_SCRIPT}")
print(f"📊 Health check URL: {HEALTH_URL}")

# اجرای بک‌اند در پس‌زمینه
backend_process = subprocess.Popen(
    [sys.executable, str(MAIN_SCRIPT)], 
    cwd=str(WORK), 
    stdout=subprocess.PIPE, 
    stderr=subprocess.STDOUT, 
    text=True, 
    env=os.environ.copy()
)

print(f"⏳ Waiting for backend to become healthy (max 120s)...")

for i in range(120):
    try:
        r = requests.get(HEALTH_URL, timeout=2)
        if r.ok:
            print('✅ Backend is healthy!')
            print('Response:', r.json())
            break
    except requests.exceptions.ConnectionError:
        # این خطا در ابتدا طبیعی است
        pass
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        pass
    time.sleep(1)
else:
    # اگر از حلقه خارج شد، یعنی بکک‌اند سالم نشده است
    print("\n❌ Backend did not become healthy in time.")
    print("Checking backend logs for errors...")
    stdout, stderr = backend_process.communicate(timeout=5)
    logs = stdout or stderr
    if logs:
        print("--- Backend Logs ---")
        print(logs)
    raise SystemExit('❌ Backend failed to start.')

In [ ]:
# 📝 Create a sample manifest (preset) for the render job
import json

# اگر درایو متصل است، از فایل‌های آن استفاده می‌کنیم
if DRIVE_ASSETS:
    audio_files = list(DRIVE_ASSETS.glob('*.mp3')) + list(DRIVE_ASSETS.glob('*.wav')) + list(DRIVE_ASSETS.glob('*.m4a'))
    image_files = list(DRIVE_ASSETS.glob('*.jpg')) + list(DRIVE_ASSETS.glob('*.jpeg')) + list(DRIVE_ASSETS.glob('*.png'))

    if not audio_files or not image_files:
        raise FileNotFoundError(
            "❌ No audio or image files found in Google Drive Assets folder. "
            f"Please upload them to {DRIVE_ASSETS}"
        )

    # استفاده از اولین فایل‌های موجود به عنوان نمونه
    audio_file = audio_files[0].name
    bg_image = image_files[0].name

    print(f"🎧 Using audio file: {audio_file}")
    print(f"🖼️ Using background image: {bg_image}")
else:
    # اگر درایو متصل نیست، از نام‌های فرضی استفاده می‌کنیم
    audio_file = "sample.mp3"
    bg_image = "sample.jpg"
    print("⚠️ Drive not connected. Using placeholder filenames.")

# ساخت یک manifest (preset) نمونه با فرمت دقیق API شما
manifest = {
  "audioSegments": [{"path": audio_file}],
  "bgPath": bg_image,
  "config": {
    "aspectRatio": "9:16",
    "captions": {
      "fontSize": 92,
      "primaryColor": "#FFFFFF",
      "highlightColor": "#FFD700",
      "position": "Bottom"
    },
    "audio": {
      "whisperModel": "medium",
      "useVAD": True
    }
  }
}

MANIFEST_PATH = pathlib.Path('/content/manifest.json')
MANIFEST_PATH.write_text(json.dumps(manifest, ensure_ascii=False, indent=2))
print(f'✅ Manifest saved to {MANIFEST_PATH}')

In [ ]:
# 🎥 Submit render job, poll for status, and download output
import urllib.request

RENDER_URL = f"{BASE_URL}/render"
STATUS_URL_TEMPLATE = BASE_URL + "/status?jobId={job_id}"

with open(MANIFEST_PATH, 'r', encoding='utf-8') as f:
    manifest_data = json.load(f)

print(f"📤 Sending render request to {RENDER_URL}...")
render_response = requests.post(RENDER_URL, json=manifest_data, timeout=120)
print('POST /render status:', render_response.status_code)
print('Response:', render_response.text)
render_response.raise_for_status()

job_data = render_response.json()
job_id = job_data.get('jobId')
if not job_id:
    raise SystemExit('❌ No jobId returned from backend')

print(f'🎬 Render job started with ID: {job_id}')

# بررسی وضعیت رندر
status_url = STATUS_URL_TEMPLATE.format(job_id=job_id)
print(f"🔍 Polling status from {status_url}...")

for i in range(900): # حداکثر 15 دقیقه صبر
    try:
        status_response = requests.get(status_url, timeout=10)
        status_response.raise_for_status()
        status_json = status_response.json()
        
        state = status_json.get('status')
        progress = status_json.get('progress', {})
        pct = progress.get('percentage', 0)
        msg = progress.get('message', '')
        
        print(f"Tick {i:03d}: Status={state}, Progress={pct}%, Message='{msg}'")
        
        if state == 'done':
            print('✅ Render complete!')
            video_url = status_json.get('videoUrl')
            if video_url:
                video_name = video_url.split('?file=')[-1]
                download_url = f"{BASE_URL}/download?file={video_name}"
                out_path = pathlib.Path(f"/content/{job_id}.mp4")
                print(f"📥 Downloading video from {download_url} to {out_path}...")
                urllib.request.urlretrieve(download_url, out_path)
                print(f'✅ Downloaded video: {out_path}')
                
                # کپی کردن به Google Drive اگر متصل باشد
                if DRIVE_OUTPUT:
                    drive_path = DRIVE_OUTPUT / out_path.name
                    shutil.copy2(out_path, drive_path)
                    print(f'✅ Also copied to Google Drive: {drive_path}')
                
                # نمایش لینک دانلود مستقیم
                from IPython.display import HTML, display
                display(HTML(f'<a href="{download_url}" download target="_blank"><button style="font-size:16px; padding:10px; background-color:#4CAF50; color:white; border:none; border-radius:5px;">🔗 دانلود ویدیو</button></a>'))
            else:
                print("❌ No videoUrl found in response.")
            break
        elif state == 'error':
            error_msg = status_json.get('error', 'Unknown error')
            raise SystemExit(f'❌ Render job failed: {error_msg}')
    except requests.exceptions.RequestException as e:
        print(f"⚠️ Could not get status: {e}")
    
    time.sleep(1)
else:
    raise SystemExit('❌ Render job did not finish in time')

print('\n🎉 Done! Check /content or your browser downloads for output.')